## Ensure that you have relevant data in data_untracked

In [1]:
import numpy as np
import pandas as pd
import sys
import os

parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from extract_data import extract 
from create_data import create_labels, create_features
# from create_data import create_features

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

### Download below required if creating footnote analysis file

In [2]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download("wordnet")
# nltk.download("averaged_perceptron_tagger")

___
## Section 1. Create Data
___

### Extract form 4 data if missing

In [3]:
data_extractor = extract.Data_Extractor()
data_extractor.create_form4()

========== Some SEC files not present ==========
=============== Begin extracting ===============
Found 76 zip file links
Downloaded: ../../data_untracked/raw/sec_submissions/interim/2024q4_form345.zip
Downloaded: ../../data_untracked/raw/sec_submissions/interim/2024q3_form345.zip


KeyboardInterrupt: 

### Create labels

In [4]:
# Create data labels if missing
label_data_creator =  create_labels.Label_Data_Creator()
label_data_creator.create_labels()

========== Required labelled files present ==========
========== Skip data label creation ==========
========== Reading successful ==========


### Create features

In [ ]:
feature_data_creator = create_features.Feature_Data_Creator()
feature_data_creator.create_features()

=== Final features file not found. Begin creating ===
=== Transaction Key file is found. Extracting ===
=== Network time_independent_features file is found. Extracting ===
column is renamed. PLEASE CHANGE THIS ASAP
=== Network time_dependent_features file is found. Extracting ===
=== Other Features Key file not found, begin creating  ===


### Create training and testing data

In [6]:
feature_data_creator.create_training_testing()

=== Training or Testing file not found. Begin creating based on quantile: 0.7
=== Saving Training and Testing ===


___
## Section 2. Modelling, Training and Validation
___

In [21]:
print("features:")
print(feature_data_creator.features)

features:
['js_bin', 's_bin', 'b_bin', 'jb_bin', 'ob_bin', 'g_bin', 'gift', 'distribution', 'charity', 'price', 'number', 'ball', 'pursuant', '10b5-1', '16b-3', 'lobbyist_score_final', 'total_senate_connections', 'total_house_connections', 'combined_seniority_score', 'PI_combined_total', 'net_trading_intensity', 'net_trading_amt', 'relative_trade_size_to_self', 'relative_trade_size_to_others', 'beneficial_ownership_score', 'js_bin', 's_bin', 'b_bin', 'jb_bin', 'ob_bin', 'g_bin', 'gift', 'distribution', 'charity', 'price', 'number', 'ball', 'pursuant', '10b5-1', '16b-3', 'lobbyist_score_final', 'total_senate_connections', 'total_house_connections', 'combined_seniority_score', 'PI_combined_total', 'net_trading_intensity', 'net_trading_amt', 'relative_trade_size_to_self', 'relative_trade_size_to_others', 'beneficial_ownership_score', 'js_bin', 's_bin', 'b_bin', 'jb_bin', 'ob_bin', 'g_bin', 'gift', 'distribution', 'charity', 'price', 'number', 'ball', 'pursuant', '10b5-1', '16b-3', 'lobbyi

___
## Compare results
___